In [1]:
import cv2
import numpy as np
import csv
import math

In [48]:
map_image = cv2.imread('perumdos.png', cv2.IMREAD_COLOR)
map_image = cv2.resize(map_image, (1920, 1080))
img = map_image.copy()

coordinates = []

with open('coordinates.csv', mode='r') as file:
  reader = csv.reader(file)
  next(reader)
  for row in reader:
    x, y = int(row[1]), int(row[2])
    coordinates.append((x, y))
    cv2.circle(img, (x, y), 5, (0, 0, 255), -1)

print(coordinates)

[(1201, 148), (1173, 156), (1133, 158), (1067, 167), (1099, 163), (1033, 174), (990, 187), (1207, 204), (1170, 220), (1107, 221), (1116, 199), (1085, 202), (1058, 221), (1042, 246), (1025, 226), (968, 213), (950, 215), (962, 194), (938, 229), (940, 258), (923, 261), (919, 234), (904, 261), (890, 271), (869, 276), (852, 283), (997, 241), (976, 247), (997, 259), (975, 263), (860, 320), (961, 287), (958, 314), (915, 357), (895, 331), (852, 367), (1220, 287), (1173, 292), (1228, 340), (1209, 351), (1178, 334), (1158, 351), (1124, 306), (1101, 309), (1046, 333), (1100, 368), (1037, 378), (1017, 342), (1239, 425), (1221, 447), (1213, 422), (1177, 434), (1113, 442), (1092, 460), (1047, 449), (1007, 442), (976, 447), (916, 424), (933, 447), (904, 444), (878, 463), (899, 486), (1249, 498), (1197, 510), (1135, 522), (1058, 536), (997, 541), (929, 557), (860, 564), (1257, 554), (1209, 555), (1131, 573), (1075, 578), (997, 586), (955, 597), (927, 603), (866, 608), (836, 608), (730, 553), (765, 504

In [40]:
def print_coordinates(event, x, y, flags, param):
	if event == cv2.EVENT_LBUTTONDOWN:
		coordinates.append((x, y))
		# print(f"[{x}, {y}],")
		cv2.circle(img, (x, y), 5, (0, 0, 255), -1)
		cv2.imshow("Image", img)

cv2.imshow("Image", img)
cv2.setMouseCallback("Image", print_coordinates)

while True:
	key = cv2.waitKey(1) & 0xFF

	if key == ord('d'):
		if len(coordinates) > 0:
			coordinates.pop()
			img = map_image.copy()
			for coord in coordinates:
				cv2.circle(img, coord, 5, (0, 0, 255), -1)
			cv2.imshow("Image", img)

	elif key == 27:
		break

cv2.destroyAllWindows()

with open('coordinates.csv', mode='w', newline='') as file:
	writer = csv.writer(file)
	writer.writerow(['No', 'X', 'Y'])
	for i, coord in enumerate(coordinates):
		writer.writerow([i, coord[0], coord[1]])

print(coordinates)

[(1201, 148), (1173, 156), (1133, 158), (1067, 167), (1099, 163), (1033, 174), (990, 187), (1207, 204), (1170, 220), (1107, 221), (1116, 199), (1085, 202), (1058, 221), (1042, 246), (1025, 226), (968, 213), (950, 215), (962, 194), (938, 229), (940, 258), (923, 261), (919, 234), (904, 261), (890, 271), (869, 276), (852, 283), (997, 241), (976, 247), (997, 259), (975, 263), (860, 320), (961, 287), (958, 314), (915, 357), (895, 331), (852, 367), (1220, 287), (1173, 292), (1228, 340), (1209, 351), (1178, 334), (1158, 351), (1124, 306), (1101, 309), (1046, 333), (1100, 368), (1037, 378), (1017, 342), (1239, 425), (1221, 447), (1213, 422), (1177, 434), (1113, 442), (1092, 460), (1047, 449), (1007, 442), (976, 447), (916, 424), (933, 447), (904, 444), (878, 463), (899, 486), (1249, 498), (1197, 510), (1135, 522), (1058, 536), (997, 541), (929, 557), (860, 564), (1257, 554), (1209, 555), (1131, 573), (1075, 578), (997, 586), (955, 597), (927, 603), (866, 608), (836, 608), (730, 553), (765, 504

In [28]:
# Fungsi untuk menghitung jarak Euclidean antara dua titik (x1, y1) dan (x2, y2)
def euclidean_distance(p1, p2):
  return math.sqrt((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2)

In [29]:
# Map Scale
x1, y1 = 658, 157
x2, y2 = 710, 148
scale = 28.8 / euclidean_distance((x1, y1), (x2, y2))

print(scale)

0.545732587858402


In [55]:
from scipy.spatial import distance_matrix

distances = distance_matrix(coordinates, coordinates)
n = len(coordinates)

mst_edges = []
visited = [False] * n
edge_costs = [float('inf')] * n
parent = [-1] * n

edge_costs[0] = 0

for _ in range(n):
    min_cost = float('inf')
    u = -1
    for v in range(n):
        if not visited[v] and edge_costs[v] < min_cost:
            min_cost = edge_costs[v]
            u = v

    visited[u] = True

    if parent[u] != -1:
        mst_edges.append((parent[u], u))

    for v in range(n):
        if not visited[v] and distances[u][v] < edge_costs[v]:
            edge_costs[v] = distances[u][v]
            parent[v] = u

img_result_prim = cv2.imread('perumdos.png')
img_result_prim = cv2.resize(img_result_prim, (1920, 1080))

for index, coord in enumerate(coordinates):
    cv2.circle(img_result_prim, tuple(coord), 5, (0, 0, 255), -1)
    cv2.putText(img_result_prim, str(index + 1), tuple(coord), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2, cv2.LINE_AA)

for edge in mst_edges:
    pt1 = tuple(coordinates[edge[0]])
    pt2 = tuple(coordinates[edge[1]])
    cv2.line(img_result_prim, pt1, pt2, (255, 0, 0), 2)

cv2.imshow("MST Prim Result", img_result_prim)
cv2.waitKey(0)
cv2.destroyAllWindows()

print("Edges in MST:")
total_weight = 0
for edge in mst_edges:
    u, v = edge
    weight = distances[u][v] * scale
    total_weight += weight
    print(f"{u} -> {v} dengan jarak {weight:.2f}")
print(f"Total jarak MST: {total_weight:.2f}")

Edges in MST:
0 -> 1 dengan jarak 29.12
1 -> 2 dengan jarak 40.05
2 -> 4 dengan jarak 34.37
4 -> 3 dengan jarak 32.25
3 -> 5 dengan jarak 34.71
3 -> 11 dengan jarak 39.36
11 -> 9 dengan jarak 29.07
9 -> 10 dengan jarak 23.77
11 -> 12 dengan jarak 33.02
12 -> 13 dengan jarak 29.68
13 -> 14 dengan jarak 26.25
14 -> 26 dengan jarak 31.76
26 -> 28 dengan jarak 18.00
26 -> 27 dengan jarak 21.84
27 -> 29 dengan jarak 16.03
29 -> 31 dengan jarak 27.78
31 -> 32 dengan jarak 27.17
27 -> 15 dengan jarak 34.93
15 -> 16 dengan jarak 18.11
16 -> 18 dengan jarak 18.44
18 -> 21 dengan jarak 19.65
15 -> 17 dengan jarak 19.92
21 -> 20 dengan jarak 27.29
20 -> 19 dengan jarak 17.26
20 -> 22 dengan jarak 19.00
22 -> 23 dengan jarak 17.20
23 -> 24 dengan jarak 21.59
24 -> 25 dengan jarak 18.38
17 -> 6 dengan jarak 28.86
25 -> 30 dengan jarak 37.85
30 -> 34 dengan jarak 36.69
34 -> 33 dengan jarak 32.80
25 -> 147 dengan jarak 38.01
30 -> 35 dengan jarak 47.68
0 -> 7 dengan jarak 56.32
7 -> 8 dengan jarak 4

In [56]:
# Kruskal's Algorithm
from scipy.spatial import distance_matrix

distances = distance_matrix(coordinates, coordinates)

edges = []
for i, coord1 in enumerate(coordinates):
  for j, coord2 in enumerate(coordinates):
    if i < j:
      dist = distances[i][j]
      edges.append((dist, i, j))

edges.sort()

parent = list(range(len(coordinates)))
rank = [0] * len(coordinates)

def find(x):
  if parent[x] != x:
    parent[x] = find(parent[x])
  return parent[x]

def union(x, y):
  rootX = find(x)
  rootY = find(y)

  if rootX != rootY:
    if rank[rootX] > rank[rootY]:
      parent[rootY] = rootX
    elif rank[rootX] < rank[rootY]:
      parent[rootX] = rootY
    else:
      parent[rootY] = rootX
      rank[rootX] += 1

mst_edges = []
total_weight = 0

for dist, u, v in edges:
  if find(u) != find(v):
    union(u, v)
    mst_edges.append((u, v, dist))
    total_weight += dist

print("Edges in MST:")
for edge in mst_edges:
  print(f"{edge[0]} -> {edge[1]} with distance {edge[2] * scale:.2f}")
print(f"Total distance of MST: {total_weight * scale:.2f}")

img_result_kruskal = cv2.imread('perumdos.png')
img_result_kruskal = cv2.resize(img_result_kruskal, (1920, 1080))

for index, node in enumerate(coordinates):
  cv2.circle(img_result_kruskal, node, 5, (0, 0, 255), -1)
  cv2.putText(img_result_kruskal, str(index+1), node, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2, cv2.LINE_AA)

for edge in mst_edges:
  pt1 = coordinates[edge[0]]
  pt2 = coordinates[edge[1]]
  cv2.line(img_result_kruskal, pt1, pt2, (255, 0, 0), 2)

cv2.imshow("MST Kruskal Result", img_result_kruskal)
cv2.waitKey(0)
cv2.destroyAllWindows()

Edges in MST:
27 -> 29 with distance 16.03
22 -> 23 with distance 17.20
19 -> 20 with distance 17.26
26 -> 28 with distance 18.00
15 -> 16 with distance 18.11
24 -> 25 with distance 18.38
16 -> 18 with distance 18.44
20 -> 22 with distance 19.00
18 -> 21 with distance 19.65
15 -> 17 with distance 19.92
89 -> 91 with distance 21.38
23 -> 24 with distance 21.59
26 -> 27 with distance 21.84
38 -> 39 with distance 21.95
119 -> 120 with distance 22.00
97 -> 98 with distance 22.02
90 -> 92 with distance 22.20
136 -> 137 with distance 22.36
132 -> 133 with distance 22.47
42 -> 43 with distance 23.19
57 -> 59 with distance 23.32
121 -> 122 with distance 23.35
9 -> 10 with distance 23.77
89 -> 125 with distance 24.74
100 -> 101 with distance 25.18
95 -> 96 with distance 25.50
48 -> 50 with distance 26.17
13 -> 14 with distance 26.25
40 -> 41 with distance 26.25
49 -> 50 with distance 26.25
93 -> 94 with distance 27.02
31 -> 32 with distance 27.17
20 -> 21 with distance 27.29
104 -> 105 with dis

In [87]:
# Displaying the result use np

image1 = img_result_prim.copy()
image2 = img_result_kruskal.copy()

height, width, _ = image1.shape

image1 = image1[0:height, 500:1400]
image2 = image2[0:height, 500:1400]

cv2.rectangle(image1, (0, 0), (image1.shape[1], 80), (255, 255, 255), -1)
cv2.rectangle(image2, (0, 0), (image2.shape[1], 80), (255, 255, 255), -1)

cv2.putText(image1, "Prim's", (int(image1.shape[1]*0.5) - 20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)
cv2.putText(image2, "Kruskal's", (int(image2.shape[1]*0.5) - 50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)

comparison = np.hstack((image1, image2))
cv2.line(comparison, (image1.shape[1], 0), (image1.shape[1], height), (0, 0, 0), 2)

cv2.imshow("Comparison", comparison)
cv2.waitKey(0)
cv2.destroyAllWindows()